# Calendar Arithmetic

## Introduction

There are many relationships between datetimes that we would like to express but that cannot be expressed with `timedelta`. One of the reasons for this is *ambiguity*. In addition to the ambiguity of absolute vs. wall times, the lengths of certain common units are poorly defined - how long is "one month from now" or "one year from now", for example.

`dateutil` provides the `relativedelta` class for calendrical operations where the duration may depend on the starting datetime, so, for example:

- "one month from now"
- "one year from now"
- "next monday"
- "until the third Thursday of the month"

A `relativedelta` is constructed of *absolute* and *relative* arguments where absolute arguments are specified with *singular* keywords (e.g. `year`, `day`) and relative arguments are specified with *plural* keywords (e.g. `years`, `days`). Be careful of confusing these, as it is a common source of bugs.

In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta, MO, TU, WE, TH, FR, SA, SU

## Absolute arguments

Adding *or* subtracting a `relativedelta` with absolute components is roughly equivalent to calling the `replace()` method on the `datetime` with the absolute components. If both absolute and relative components exist, the absolute components are applied first.

The absolute arguments (in order of application) are: `year`, `month`, `day`, `minute`, `second`, `microsecond`.

If `relativedelta` arithmetic would create an invalid date, it will fall back to the most recent valid date:

In [2]:
datetime(2015, 2, 1) + relativedelta(day=30)

datetime.datetime(2015, 2, 28, 0, 0)

In [3]:
datetime(2016, 2, 1) + relativedelta(day=30)

datetime.datetime(2016, 2, 29, 0, 0)

### Exercise: Try out a few absolute deltas

Examples:
- March 3rd of this year
- This date in 1951
- Today at 12:15

In [4]:
# As a convenience, you can use `dateutil.utils.today` to get the current day
from dateutil.utils import today

## Relative arguments
Relative arguments represent a calendar offset from the given datetime, and is applied *after* its absolute equivalents, from largest to smallest.

The relative arguments are (in order of application): `years`, `months`, `days`, `minutes`, `seconds`, `microseconds`.

One thing to note is that because invalid dates fall back to valid ones, `relativedelta` arithmetic is a *lossy* operation:

In [5]:
dt = datetime(2020, 3, 31)
dt + relativedelta(months=1)

datetime.datetime(2020, 4, 30, 0, 0)

In [6]:
(dt + relativedelta(months=1)) + relativedelta(months=1)

datetime.datetime(2020, 5, 30, 0, 0)

To avoid this, if you want to apply multiple `relativedelta` operations to a datetime *without* loss, combine them first and then apply:

In [7]:
dt + (relativedelta(months=1) * 2)

datetime.datetime(2020, 5, 31, 0, 0)

## Weekdays

A special kind of argument is the `weekday` argument, which specifies an offset clipped to the nearest specified weekday. Like with the absolute arguments, weekdays are not affected by multiplication, and they have the same effect whether you add or subtract them:

In [8]:
datetime(1975, 12, 30) + relativedelta(weekday=FR)

datetime.datetime(1976, 1, 2, 0, 0)

In [9]:
datetime(1975, 12, 30) - relativedelta(weekday=FR)

datetime.datetime(1976, 1, 2, 0, 0)

Unlike absolute arguments, however, they carry a direction and magnitude of their own:

In [10]:
# The first Friday on or before 1975-12-30
datetime(1975, 12, 30) + relativedelta(weekday=FR(-1))

datetime.datetime(1975, 12, 26, 0, 0)

In [11]:
# The second Friday on or after 1975-12-30
datetime(1975, 12, 30) + relativedelta(weekday=FR(2))

datetime.datetime(1976, 1, 9, 0, 0)

Note that this uses a 1-based index, and the original datetime *itself* counts, so for example 1975-12-30 is a Tuesday, so both `TU(+1)` and `TU(-1)` resolve to the same datetime:

In [12]:
datetime(1975, 12, 30) + relativedelta(weekday=TU(-1))

datetime.datetime(1975, 12, 30, 0, 0)

In [13]:
datetime(1975, 12, 30) + relativedelta(weekday=TU(+1))

datetime.datetime(1975, 12, 30, 0, 0)

### Exercise: Try out some relative and weekday deltas

Examples:
- 4 weeks from now
- 1 month from now
- The Wednesday *after* next Wednesday
- Last Friday

## Combinations

The most interesting use of `relativedelta` is when you combine absolute and relative deltas, so for example, we can write a relativedelta that gives us the beginning of the next month by combining `months` and `day`:

In [14]:
next_month = relativedelta(months=1, day=1)

examples = [
    datetime(2019, 2, 7),
    datetime(2019, 3, 1),
    datetime(2019, 1, 31),
]

for dt in examples:
    print(f"{dt} ---> {dt + next_month}")

2019-02-07 00:00:00 ---> 2019-03-01 00:00:00
2019-03-01 00:00:00 ---> 2019-04-01 00:00:00
2019-01-31 00:00:00 ---> 2019-02-01 00:00:00


- As a bonus exercise, try writing a relative delta that gives you the end of the *current* month. For one way to do this, see the `rd_answers.end_of_month`.

Another example of how this can be useful is that you can write a `relativedelta` that functions that express holidays that always fall on a specific day of the week. For example, in the US Mother's day is always the second Sunday in May, we can write a `relativedelta` that, when added to any date, gives Mother's day for that year:

In [15]:
mothers_day = relativedelta(
    month=5, day=1,     # Start at the beginning of May
    weekday=SU(2),      # Jump go to the second Sunday on or after
)

In [16]:
for year in (2018, 2019, 2020):
    print(datetime(year, 1, 1) + mothers_day)

2018-05-13 00:00:00
2019-05-12 00:00:00
2020-05-10 00:00:00


### Exercise: Implement a `tzinfo` that implements the current US DST rules

In the U.S., daylight saving time starts on the second Sunday in March and ends on the first Sunday in November, with time changes taking place at 2:00 AM local time. In practice, it is unnecessary and a bad idea to implement a `tzinfo` object encoding this rules rather than using the IANA database (accessed through `dateutil.tz.gettz`) or one of the other `dateutil` time zones (such as `dateutil.tz.tzrange`). However, as a learning exercise, try using `relativedelta` to implement a `tzinfo` zone representing Eastern Time, which is:

- `'EST'` with offset `-05:00` before 2 AM the second Sunday in March
- `'EDT'` with offset `-04:00` before 2 AM the first Sunday in November

**Note**: Do not bother implementing proper `fold` handling unless you finish early.

In [17]:
from datetime import timedelta, tzinfo

class Eastern(tzinfo):
    def __init__(self):
        super().__init__()

    def __repr__(self):
        return f"{self.__class__.__name__}()"
    
    def tzname(self, dt):
        pass
        
    def utcoffset(self, dt):
        pass
    
    def dst(self, dt):
        pass


In [18]:
from helper_functions import print_dt_tzinfo
from rd_answers import Eastern as _Eastern

#################
# Remove this part before testing
Eastern = _Eastern
#################

# Tests
print_dt_tzinfo(datetime(2019, 3, 10, 1, 59, tzinfo=Eastern()))
print_dt_tzinfo(datetime(2019, 3, 10, 3,  0, tzinfo=Eastern()))
print("")
print_dt_tzinfo(datetime(2019, 11, 3, 1, 59, fold=0, tzinfo=Eastern()))
print_dt_tzinfo(datetime(2019, 11, 3, 1, 59, fold=1, tzinfo=Eastern()))
print_dt_tzinfo(datetime(2019, 11, 3, 2,  0, tzinfo=Eastern()))

2019-03-10 01:59:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h
2019-03-10 03:00:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h

2019-11-03 01:59:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h
2019-11-03 01:59:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h
2019-11-03 02:00:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


In [19]:
from dateutil import tz
tz.datetime_exists(datetime(2019, 3, 10, 2, 30, tzinfo=Eastern()))  # Should be False

False

In [20]:
# Will be True if fold support is handled correctly
tz.datetime_ambiguous(datetime(2019, 11, 3, 2, 30, tzinfo=Eastern()))

False